# Part 1

In [3]:
import numpy as np
from collections import defaultdict
import os

## The IntCode computer from Day 9
### Modified to play the game the elves have sent!

In [2]:
class intCodeComputer:
    def __init__(self, filename):
        self.program = defaultdict(lambda:'0')
        with open(filename, 'r') as f:
            line = f.readline()
        for i, code in enumerate(line.split(',')):
            self.program[i] = code

        self.memory_location = 0  
        self.relative_base = 0
        self.intcode_dict = {'01':4, '02':4, '03':2, '04':2, '05':3, '06':3, '07':4, '08':4, 
                             '09':2, '99':0}
        
        self.screenOutputs = []
        self.nBlockTiles = 0
        
    def process_parameter(self, parameter_mode, offset):
        if(parameter_mode == '0'):
            return self.program[int(self.program[self.memory_location + offset])]
        elif(parameter_mode == '1'):
            return self.program[self.memory_location + offset]
        elif(parameter_mode == '2'):
            relative_mode_parameter = int(self.program[self.memory_location + offset])
            return self.program[self.relative_base + relative_mode_parameter]
    
    # Renders game screen
    def renderOutput(self):
        x, y, tileId = self.screenOutputs
        if tileId == '2':
            self.nBlockTiles += 1
        
            
    def execute_program(self, user_input = None):
        self.user_input = user_input
        while(True):
            p_opcode = self.program[self.memory_location]
            p_opcode = p_opcode.zfill(5)
            a, b, c, de = p_opcode[0], p_opcode[1], p_opcode[2], p_opcode[3:]
            #print(a, b, c, de)

            if(de == '01'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2) 
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                self.program[parameter3] = str(int(parameter1) + int(parameter2))
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '02'):
                parameter1 = self.process_parameter(c, 1) 
                parameter2 = self.process_parameter(b, 2)
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                self.program[parameter3] = str(int(parameter1) * int(parameter2))
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '03'):
                if(c == '0'):
                    parameter1 = int(self.program[self.memory_location + 1])
                elif(c == '1'):
                    print("I shouldn't be here")
                    parameter1 = int(self.memory_location + 1)                    
                elif(c == '2'):
                    parameter1 = self.relative_base + int(self.program[self.memory_location + 1])
                    
                self.program[parameter1] = self.user_input                    
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '04'):
                # Instead of printing to console, we now render
                # output to a screen
                
                parameter1 = self.process_parameter(c, 1)
                self.screenOutputs.append(parameter1)
                #print("Here: {}".format(parameter1))
                
                if(len(self.screenOutputs) == 3):
                    self.renderOutput()
                    self.screenOutputs = []
                
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '05'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2)
                # print("Got here. Parameter 1, 2: {}, {}".format(parameter1, parameter2))

                if(int(parameter1) != 0):
                    self.memory_location = int(parameter2)
                else:        
                    self.memory_location += self.intcode_dict[de]

                continue

            if(de == '06'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2)

                if(int(parameter1) == 0):
                    self.memory_location = int(parameter2)
                else:        
                    self.memory_location += self.intcode_dict[de]

                continue

            if(de == '07'):
                parameter1 = int(self.process_parameter(c, 1))
                parameter2 = int(self.process_parameter(b, 2))
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base

                if(parameter1 < parameter2):
                    self.program[parameter3] = '1'
                else:        
                    self.program[parameter3] = '0'

                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '08'):
                parameter1 = int(self.process_parameter(c, 1))
                parameter2 = int(self.process_parameter(b, 2))
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                if(parameter1 == parameter2):
                    self.program[parameter3] = '1'
                else:        
                    self.program[parameter3] = '0'

                self.memory_location += self.intcode_dict[de]
                continue
            
            if(de == '09'):
                parameter1 = self.process_parameter(c, 1)
                self.relative_base += int(parameter1)
                self.memory_location += self.intcode_dict[de]
                continue                

            if(de == '99'):
                print("Number of blocks rendered: {}".format(self.nBlockTiles))
                break

In [3]:
input_file = 'day13_input'
myComputer = intCodeComputer(input_file)
myComputer.execute_program()

Number of blocks rendered: 326


# Part 2

# ModIfying the IntCode computer a bit

### I cheated a bit and didn't actually render the game in the first half, so I'm going to try do that now.

In [4]:
# Bit the bullet and installed the keyboard module 
# to read joystick inputs for the arcade machine
import getch

In [6]:
class intCodeComputer:
    def __init__(self, filename):
        self.program = defaultdict(lambda:'0')
        with open(filename, 'r') as f:
            line = f.readline()
        for i, code in enumerate(line.split(',')):
            self.program[i] = code

        self.memory_location = 0  
        self.relative_base = 0
        self.intcode_dict = {'01':4, '02':4, '03':2, '04':2, '05':3, '06':3, '07':4, '08':4, 
                             '09':2, '99':0}
        
        self.gameScreen = np.zeros((2000, 2000), dtype='str')
        self.screenMax_x = 0
        self.screenMax_y = 0
        self.screenOutputs = []
        self.nBlockTiles = 0
        
        self.gameItemId = {'0':" ", '1':"#", '2':'=', '3':"__", '4':"o"}
        
        self.user_input = 0
    
    def process_parameter(self, parameter_mode, offset):
        if(parameter_mode == '0'):
            return self.program[int(self.program[self.memory_location + offset])]
        elif(parameter_mode == '1'):
            return self.program[self.memory_location + offset]
        elif(parameter_mode == '2'):
            relative_mode_parameter = int(self.program[self.memory_location + offset])
            return self.program[self.relative_base + relative_mode_parameter]
    
    # Renders game screen
    def renderOutput(self):
        x, y, tileId = self.screenOutputs
        self.gameScreen[int(x), int(y)] = self.gameItemId[tileId]
        
        if(int(x) > self.screenMax_x):
            self.screenMax_x = int(x)
        if(int(y) > self.screenMax_y):
            self.screenMax_y = int(y)
        # os.system('clear')
        for i in range(self.screenMax_y+1):
            print(''.join(self.gameScreen[:self.screenMax_x+1, i]))
        
        
        
    def execute_program(self, input_quarters=0):
        self.program[self.memory_location] = str(input_quarters)
        while(True):
            p_opcode = self.program[self.memory_location]
            p_opcode = p_opcode.zfill(5)
            a, b, c, de = p_opcode[0], p_opcode[1], p_opcode[2], p_opcode[3:]
            #print(a, b, c, de)

            if(de == '01'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2) 
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                self.program[parameter3] = str(int(parameter1) + int(parameter2))
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '02'):
                parameter1 = self.process_parameter(c, 1) 
                parameter2 = self.process_parameter(b, 2)
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                self.program[parameter3] = str(int(parameter1) * int(parameter2))
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '03'):
                if(c == '0'):
                    parameter1 = int(self.program[self.memory_location + 1])
                elif(c == '1'):
                    print("I shouldn't be here")
                    parameter1 = int(self.memory_location + 1)                    
                elif(c == '2'):
                    parameter1 = self.relative_base + int(self.program[self.memory_location + 1])
                    
                self.program[parameter1] = self.user_input                    
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '04'):
                # Instead of printing to console, we now render
                # output to a screen
                
                parameter1 = self.process_parameter(c, 1)
                self.screenOutputs.append(parameter1)
                #print("Here: {}".format(parameter1))
                
                if(len(self.screenOutputs) == 3):
                    self.renderOutput()
                    self.screenOutputs = []
                
                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '05'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2)
                # print("Got here. Parameter 1, 2: {}, {}".format(parameter1, parameter2))

                if(int(parameter1) != 0):
                    self.memory_location = int(parameter2)
                else:        
                    self.memory_location += self.intcode_dict[de]

                continue

            if(de == '06'):
                parameter1 = self.process_parameter(c, 1)
                parameter2 = self.process_parameter(b, 2)

                if(int(parameter1) == 0):
                    self.memory_location = int(parameter2)
                else:        
                    self.memory_location += self.intcode_dict[de]

                continue

            if(de == '07'):
                parameter1 = int(self.process_parameter(c, 1))
                parameter2 = int(self.process_parameter(b, 2))
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base

                if(parameter1 < parameter2):
                    self.program[parameter3] = '1'
                else:        
                    self.program[parameter3] = '0'

                self.memory_location += self.intcode_dict[de]
                continue

            if(de == '08'):
                parameter1 = int(self.process_parameter(c, 1))
                parameter2 = int(self.process_parameter(b, 2))
                parameter3 = int(self.program[self.memory_location+3])
                if(a == '2'):
                    parameter3 += self.relative_base
                    
                if(parameter1 == parameter2):
                    self.program[parameter3] = '1'
                else:        
                    self.program[parameter3] = '0'

                self.memory_location += self.intcode_dict[de]
                continue
            
            if(de == '09'):
                parameter1 = self.process_parameter(c, 1)
                self.relative_base += int(parameter1)
                self.memory_location += self.intcode_dict[de]
                continue                

            if(de == '99'):
                print("Number of blocks rendered: {}".format(self.nBlockTiles))
                break

In [ ]:
input_file = 'day13_input'
myComputer = intCodeComputer(input_file)
myComputer.execute_program(2) # Free quarters - yay!